# Deep cortical reinforcement learning: Policy gradients + Q-learning + GAN


In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [9]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [10]:
import gym

## Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [11]:
env.reset()
batch = []
for _ in range(1111):
    #env.render()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action) # take a random action
    batch.append([action, state, reward, done, info])
    #print('state, action, reward, done, info:', state, action, reward, done, info)
    if done:
        env.reset()

To shut the window showing the simulation, use `env.close()`.

In [12]:
# env.close()

If you ran the simulation above, we can look at the rewards:

In [13]:
batch[0], 
batch[0][1].shape, state.shape

((4,), (4,))

In [14]:
import numpy as np
actions = np.array([each[0] for each in batch])
states = np.array([each[1] for each in batch])
rewards = np.array([each[2] for each in batch])
dones = np.array([each[3] for each in batch])
infos = np.array([each[4] for each in batch])

In [15]:
# print(rewards[-20:])
print('shapes:', np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
print('dtypes:', np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
print('states:', np.max(np.array(states)), np.min(np.array(states)))
print('actions:', np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
print('rewards:', np.max(np.array(rewards)), np.min(np.array(rewards)))

shapes: (1111,) (1111, 4) (1111,) (1111,)
dtypes: float64 float64 int64 bool
states: 2.6213634468955913 -2.831909608650985
actions: 1 0
rewards: 1.0 1.0


In [16]:
actions[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [17]:
rewards[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [18]:
# import numpy as np
def sigmoid(x, derivative=False):
  return x*(1-x) if derivative else 1/(1+np.exp(-x))

In [19]:
sigmoid(np.max(np.array(rewards))), sigmoid(np.min(np.array(rewards)))

(0.7310585786300049, 0.7310585786300049)

In [20]:
print('rewards:', np.max(np.array(rewards))/100, np.min(np.array(rewards))/100)

rewards: 0.01 0.01


The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [21]:
# Data of the model
def model_input(state_size, lstm_size, batch_size=1):
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    reward = tf.placeholder(tf.float32, [], name='reward')
    # GRU: Gated Recurrent Units
    gru = tf.nn.rnn_cell.GRUCell(lstm_size) # hidden size
    cell = tf.nn.rnn_cell.MultiRNNCell([gru], state_is_tuple=False)
    g_initial_state = cell.zero_state(batch_size, tf.float32) # feedback or lateral/recurrent connection from output
    d_initial_state = cell.zero_state(batch_size, tf.float32) # feedback or lateral/recurrent connection from output
    return states, actions, targetQs, reward, cell, g_initial_state, d_initial_state

In [22]:
# How to use batch-norm
#   x_norm = tf.layers.batch_normalization(x, training=training)

#   # ...

#   update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
#   with tf.control_dependencies(update_ops):
#     train_op = optimizer.minimize(loss)

In [23]:
# training: Either a Python boolean, or a TensorFlow boolean scalar tensor (e.g. a placeholder). 
# Whether to return the output in: 
# training mode (normalized with statistics of the current batch) or 
# inference mode (normalized with moving statistics). 
# NOTE: make sure to set this parameter correctly, or else your training/inference will not work properly.

In [24]:
# MLP & Conv
# # Generator/Controller: Generating/prediting the actions
# def generator(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
#     with tf.variable_scope('generator', reuse=reuse):
#         # First fully connected layer
#         h1 = tf.layers.dense(inputs=states, units=hidden_size)
#         bn1 = tf.layers.batch_normalization(h1, training=training)        
#         nl1 = tf.maximum(alpha * bn1, bn1)
        
#         # Second fully connected layer
#         h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
#         bn2 = tf.layers.batch_normalization(h2, training=training)        
#         nl2 = tf.maximum(alpha * bn2, bn2)
        
#         # Output layer
#         logits = tf.layers.dense(inputs=nl2, units=action_size)        
#         #predictions = tf.nn.softmax(logits)

#         # return actions logits
#         return logits

In [25]:
# RNN generator or sequence generator
def generator(states, initial_state, cell, lstm_size, num_classes, reuse=False): 
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        inputs = tf.layers.dense(inputs=states, units=lstm_size)
        print(states.shape, inputs.shape)
        
        # with tf.variable_scope('dynamic_rnn_', reuse=tf.AUTO_REUSE):
        # dynamic means adapt to the batch_size
        inputs_rnn = tf.reshape(inputs, [1, -1, lstm_size]) # NxH -> 1xNxH
        print(inputs_rnn.shape, initial_state.shape)
        outputs_rnn, final_state = tf.nn.dynamic_rnn(cell=cell, inputs=inputs_rnn, initial_state=initial_state)
        print(outputs_rnn.shape, final_state.shape)
        outputs = tf.reshape(outputs_rnn, [-1, lstm_size]) # 1xNxH -> NxH
        print(outputs.shape)

        # Last fully connected layer
        logits = tf.layers.dense(inputs=outputs, units=num_classes)
        print(logits.shape)
        #predictions = tf.nn.softmax(logits)
        
        # logits are the action logits
        return logits, final_state

In [26]:
# MLP & Conv
# # Discriminator/Dopamine: Reward function/planner/naviator/advisor/supervisor/cortical columns
# def discriminator(states, actions, hidden_size, reuse=False, alpha=0.1, training=False):
#     with tf.variable_scope('discriminator', reuse=reuse):
#         # Fusion/merge states and actions/ SA/ SM
#         x_fused = tf.concat(axis=1, values=[states, actions])
        
#         # First fully connected layer
#         h1 = tf.layers.dense(inputs=x_fused, units=hidden_size)
#         bn1 = tf.layers.batch_normalization(h1, training=training)        
#         nl1 = tf.maximum(alpha * bn1, bn1)
        
#         # Second fully connected layer
#         h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
#         bn2 = tf.layers.batch_normalization(h2, training=training)        
#         nl2 = tf.maximum(alpha * bn2, bn2)
        
#         # Output layer
#         logits = tf.layers.dense(inputs=nl2, units=1)        
#         #predictions = tf.nn.softmax(logits)

#         # return rewards logits
#         return logits

In [27]:
# RNN generator or sequence generator
def discriminator(states, actions, initial_state, cell, lstm_size, reuse=False): 
    with tf.variable_scope('discriminator', reuse=reuse):
        # Fusion/merge states and actions/ SA/ SM
        x_fused = tf.concat(axis=1, values=[states, actions])
        
        # First fully connected layer
        inputs = tf.layers.dense(inputs=x_fused, units=lstm_size)
        print(states.shape, inputs.shape)
        
        # with tf.variable_scope('dynamic_rnn_', reuse=tf.AUTO_REUSE):
        inputs_rnn = tf.reshape(inputs, [1, -1, lstm_size]) # NxH -> 1xNxH
        print(inputs_rnn.shape, initial_state.shape)
        outputs_rnn, final_state = tf.nn.dynamic_rnn(cell=cell, inputs=inputs_rnn, initial_state=initial_state)
        print(outputs_rnn.shape, final_state.shape)
        outputs = tf.reshape(outputs_rnn, [-1, lstm_size]) # 1xNxH -> NxH
        print(outputs.shape)

        # Last fully connected layer
        logits = tf.layers.dense(inputs=outputs, units=1)
        print(logits.shape)
        #predictions = tf.nn.softmax(logits)
        
        # logits are the action logits
        return logits, final_state

In [28]:
def model_loss(action_size, hidden_size, states, actions, targetQs, reward,
               cell, g_initial_state, d_initial_state):
    # G/Actor
    #actions_logits = generator(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_logits, g_final_state = generator(states=states, num_classes=action_size, 
                                              cell=cell, initial_state=g_initial_state, lstm_size=hidden_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    neg_log_prob_actions = tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels)
    g_loss = tf.reduce_mean(neg_log_prob_actions * targetQs)
    
    # D/Critic
    #Qs_logits = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states)
    Qs_logits, d_final_state = discriminator(states=states, actions=actions_logits, 
                                             cell=cell, initial_state=d_initial_state, lstm_size=hidden_size)
    rewards = reward * tf.ones_like(targetQs)
    d_lossR = tf.reduce_mean(tf.square(tf.nn.sigmoid(tf.reshape(Qs_logits, [-1])) - rewards))
    d_lossR_sigm = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=tf.reshape(Qs_logits, [-1]),
                                                                          labels=rewards))
    d_lossQ = tf.reduce_mean(tf.square(tf.reshape(Qs_logits, [-1]) - targetQs))
    d_lossQ_sigm = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=tf.reshape(Qs_logits, [-1]),
                                                                          labels=tf.nn.sigmoid(targetQs)))
    #d_loss = d_lossR_sigm + d_lossQ_sigm + d_lossR + d_lossQ
    d_loss = d_lossR_sigm + d_lossQ

    return actions_logits, Qs_logits, g_final_state, d_final_state, g_loss, d_loss, d_lossR, d_lossQ, d_lossR_sigm, d_lossQ_sigm

In [29]:
# Optimizating/training/learning G & D
def model_opt(g_loss, d_loss, learning_rate):
    """
    Get optimization operations in order
    :param g_loss: Generator loss Tensor for action prediction
    :param d_loss: Discriminator loss Tensor for reward prediction for generated/prob/logits action
    :param learning_rate: Learning Rate Placeholder
    :return: A tuple of (qfunction training, generator training, discriminator training)
    """
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize RNN
    # g_grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(g_loss, g_vars), clip_norm=5) # usually around 1-5
    # d_grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(d_loss, d_vars), clip_norm=5) # usually around 1-5
    g_grads=tf.gradients(g_loss, g_vars)
    d_grads=tf.gradients(d_loss, d_vars)
    g_opt = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads_and_vars=zip(g_grads, g_vars))
    d_opt = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads_and_vars=zip(d_grads, d_vars))
    
    # # Optimize MLP & CNN
    # with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
    #     g_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)
    #     d_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)

    return g_opt, d_opt

In [30]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs, self.reward, cell, self.g_initial_state, self.d_initial_state = model_input(
            state_size=state_size, lstm_size=hidden_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.Qs_logits, self.g_final_state, self.d_final_state, self.g_loss, self.d_loss, self.d_lossR, self.d_lossQ, self.d_lossR_sigm, self.d_lossQ_sigm = model_loss(
            action_size=action_size, hidden_size=hidden_size,
            states=self.states, actions=self.actions, cell=cell,
            targetQs=self.targetQs, reward=self.reward,  
            g_initial_state=self.g_initial_state, d_initial_state=self.d_initial_state)
        
        # Update the model: backward pass and backprop
        self.g_opt, self.d_opt = model_opt(g_loss=self.g_loss, d_loss=self.d_loss, learning_rate=learning_rate)

In [31]:
from collections import deque

class Memory():    
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
        self.states = deque(maxlen=max_size)

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [32]:
print('state size:{}'.format(states.shape), 
      'actions:{}'.format(actions.shape)) 
print('action size:{}'.format(np.max(actions) - np.min(actions)+1))

state size:(1111, 4) actions:(1111,)
action size:2


In [33]:
# Training parameters
# Network parameters
state_size = 4                 # number of units for the input state/observation -- simulation
action_size = 2                # number of units for the output actions -- simulation
hidden_size = 64               # number of units in each Q-network hidden layer -- simulation
batch_size = 32                # number of samples in the memory/ experience as mini-batch size
learning_rate = 0.001          # learning rate for adam

In [34]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

# Init the memory
memory = Memory(max_size=batch_size)

(?, 4) (?, 64)
(1, ?, 64) (1, 64)
(1, ?, 64) (1, 64)
(?, 64)
(?, 2)
(?, 4) (?, 64)
(1, ?, 64) (1, 64)
(1, ?, 64) (1, 64)
(?, 64)
(?, 1)


In [35]:
state = env.reset()
for _ in range(batch_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done)])
    state = next_state
    if done is True:
        state = env.reset()

In [36]:
memory.buffer[0]

[array([-0.00183902, -0.00193909,  0.00719643, -0.01100595]),
 0,
 array([-0.0018778 , -0.1971635 ,  0.00697631,  0.28393882]),
 1.0,
 0.0]

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [37]:
from collections import deque
episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
saver = tf.train.Saver()
rewards_list, g_loss_list, d_loss_list = [], [], []
rates_list, d_lossR_list, d_lossQ_list = [], [], []
d_lossRsigm_list, d_lossQsigm_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Training episodes/epochs
    for ep in range(11111):
        batch = [] # every data batch
        total_reward = 0
        state = env.reset() # env first state
        g_initial_state = sess.run(model.g_initial_state)
        d_initial_state = sess.run(model.d_initial_state)

        # Training steps/batches
        while True:
            action_logits, g_final_state, d_final_state = sess.run(
                fetches=[model.actions_logits, model.g_final_state, model.d_final_state], 
                feed_dict={model.states: np.reshape(state, [1, -1]),
                           model.g_initial_state: g_initial_state,
                           model.d_initial_state: d_initial_state})
            action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done)])
            memory.states.append([g_initial_state, g_final_state,
                                  d_initial_state, d_final_state])
            total_reward += reward
            g_initial_state = g_final_state
            d_initial_state = d_final_state
            state = next_state
            if done is True:
                break

        # Episode total reward and success rate/prob
        episode_reward.append(total_reward) # stopping criteria
        rate = total_reward/ 500 # success is 500 points: 0-1
            
        # Training
        batch = memory.buffer
        states = np.array([each[0] for each in batch])
        actions = np.array([each[1] for each in batch])
        next_states = np.array([each[2] for each in batch])
        rewards = np.array([each[3] for each in batch])
        dones = np.array([each[4] for each in batch])
        rnn_states = memory.states
        g_initial_states = np.array([each[0] for each in rnn_states])
        g_final_states = np.array([each[1] for each in rnn_states])
        d_initial_states = np.array([each[2] for each in rnn_states])
        d_final_states = np.array([each[3] for each in rnn_states])
        nextQs_logits = sess.run(fetches = model.Qs_logits,
                                 feed_dict = {model.states: next_states, 
                                              model.g_initial_state: g_final_states[0].reshape([1, -1]),
                                              model.d_initial_state: d_final_states[0].reshape([1, -1])})
        nextQs = nextQs_logits.reshape([-1]) * (1-dones) # exploit
        #print(nextQs.shape, nextQs_logits.shape, dones.shape, rewards.shape)
        targetQs = rewards + (0.99 * nextQs)
        #print(targetQs.shape, rewards.shape, nextQs.shape)
        g_loss, d_loss, d_lossR, d_lossQ, d_lossRsigm, d_lossQsigm, _, _ = sess.run(
            fetches=[model.g_loss, model.d_loss, 
                     model.d_lossR, model.d_lossQ, 
                     model.d_lossR_sigm, model.d_lossQ_sigm,
                     model.g_opt, model.d_opt], 
            feed_dict = {model.states: states, model.actions: actions,
                         model.reward: rate, model.targetQs: targetQs,
                         model.g_initial_state: g_initial_states[0].reshape([1, -1]),
                         model.d_initial_state: d_initial_states[0].reshape([1, -1])})
 
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'rate:{:.4f}'.format(rate),
              'gloss:{:.4f}'.format(g_loss),
              'dloss:{:.4f}'.format(d_loss),
              'dlossR:{:.4f}'.format(d_lossR),
              'dlossQ:{:.4f}'.format(d_lossQ),
              'dlossRsigm:{:.4f}'.format(d_lossRsigm),
              'dlossQsigm:{:.4f}'.format(d_lossQsigm))
        # Ploting out
        rewards_list.append([ep, np.mean(episode_reward)])
        rates_list.append([ep, rate])
        g_loss_list.append([ep, g_loss])
        d_loss_list.append([ep, d_loss])
        d_lossR_list.append([ep, d_lossR])
        d_lossQ_list.append([ep, d_lossQ])
        d_lossRsigm_list.append([ep, d_lossRsigm])
        d_lossQsigm_list.append([ep, d_lossQsigm])
        # Break episode/epoch loop
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:39.0000 rate:0.0780 gloss:0.6308 dloss:1.6666 dlossR:0.1630 dlossQ:1.0036 dlossRsigm:0.6630 dlossQsigm:0.7097
Episode:1 meanR:78.5000 rate:0.2360 gloss:0.6941 dloss:1.7042 dlossR:0.0749 dlossQ:1.0005 dlossRsigm:0.7037 dlossQsigm:0.6841
Episode:2 meanR:68.6667 rate:0.0980 gloss:0.6337 dloss:1.6769 dlossR:0.1524 dlossQ:1.0022 dlossRsigm:0.6747 dlossQsigm:0.7039
Episode:3 meanR:69.2500 rate:0.1420 gloss:0.5623 dloss:1.6525 dlossR:0.1024 dlossQ:1.0111 dlossRsigm:0.6414 dlossQsigm:0.7269
Episode:4 meanR:71.4000 rate:0.1600 gloss:0.7452 dloss:1.7364 dlossR:0.1390 dlossQ:0.9963 dlossRsigm:0.7402 dlossQsigm:0.6619
Episode:5 meanR:84.0000 rate:0.2940 gloss:0.4946 dloss:1.6689 dlossR:0.0225 dlossQ:1.0159 dlossRsigm:0.6530 dlossQsigm:0.7423
Episode:6 meanR:85.7143 rate:0.1920 gloss:0.4015 dloss:1.6258 dlossR:0.0454 dlossQ:1.0329 dlossRsigm:0.5929 dlossQsigm:0.7703
Episode:7 meanR:93.0000 rate:0.2880 gloss:0.3234 dloss:1.6663 dlossR:0.0073 dlossQ:1.0498 dlossRsigm:0.6165 dlossQsigm

Episode:65 meanR:220.6212 rate:0.4420 gloss:0.3450 dloss:1.7137 dlossR:0.0014 dlossQ:1.0245 dlossRsigm:0.6892 dlossQsigm:0.7629
Episode:66 meanR:221.4179 rate:0.5480 gloss:1.0894 dloss:1.9189 dlossR:0.0804 dlossQ:1.0090 dlossRsigm:0.9099 dlossQsigm:0.3093
Episode:67 meanR:221.6765 rate:0.4780 gloss:0.5802 dloss:1.6945 dlossR:0.0040 dlossQ:0.9942 dlossRsigm:0.7003 dlossQsigm:0.6567
Episode:68 meanR:222.6522 rate:0.5780 gloss:0.6951 dloss:1.6703 dlossR:0.0004 dlossQ:0.9886 dlossRsigm:0.6817 dlossQsigm:0.6258
Episode:69 meanR:222.5571 rate:0.4320 gloss:0.8429 dloss:1.7459 dlossR:0.0389 dlossQ:0.9820 dlossRsigm:0.7639 dlossQsigm:0.5609
Episode:70 meanR:222.5211 rate:0.4400 gloss:0.6884 dloss:1.7019 dlossR:0.0144 dlossQ:0.9870 dlossRsigm:0.7150 dlossQsigm:0.6351
Episode:71 meanR:222.5000 rate:0.4420 gloss:0.8202 dloss:1.7649 dlossR:0.0472 dlossQ:0.9795 dlossRsigm:0.7854 dlossQsigm:0.5270
Episode:72 meanR:223.3425 rate:0.5680 gloss:1.0805 dloss:1.9058 dlossR:0.0795 dlossQ:0.9921 dlossRsigm:0

Episode:129 meanR:277.4800 rate:0.5900 gloss:0.6331 dloss:1.8951 dlossR:0.0580 dlossQ:1.0501 dlossRsigm:0.8450 dlossQsigm:0.3144
Episode:130 meanR:278.5600 rate:0.5420 gloss:0.5960 dloss:2.1946 dlossR:0.1246 dlossQ:1.0896 dlossRsigm:1.1050 dlossQsigm:0.2205
Episode:131 meanR:279.6200 rate:0.5520 gloss:0.5737 dloss:2.2136 dlossR:0.1218 dlossQ:1.1055 dlossRsigm:1.1081 dlossQsigm:0.2120
Episode:132 meanR:280.3800 rate:0.5280 gloss:0.6075 dloss:2.2911 dlossR:0.1487 dlossQ:1.0742 dlossRsigm:1.2169 dlossQsigm:0.1909
Episode:133 meanR:281.4000 rate:0.5620 gloss:0.4443 dloss:2.2932 dlossR:0.1350 dlossQ:1.0808 dlossRsigm:1.2124 dlossQsigm:0.1669
Episode:134 meanR:281.6000 rate:0.5780 gloss:0.4432 dloss:2.2650 dlossR:0.1231 dlossQ:1.0963 dlossRsigm:1.1687 dlossQsigm:0.1675
Episode:135 meanR:281.9600 rate:0.5660 gloss:0.4688 dloss:2.3300 dlossR:0.1411 dlossQ:1.0594 dlossRsigm:1.2706 dlossQsigm:0.1459
Episode:136 meanR:281.7400 rate:0.4500 gloss:0.5910 dloss:1.8463 dlossR:0.1078 dlossQ:0.9010 dlos

Episode:193 meanR:195.7000 rate:0.3560 gloss:0.2273 dloss:5.3975 dlossR:0.4113 dlossQ:1.5489 dlossRsigm:3.8485 dlossQsigm:0.0544
Episode:194 meanR:194.5700 rate:0.2940 gloss:0.1266 dloss:6.3587 dlossR:0.4953 dlossQ:2.0018 dlossRsigm:4.3569 dlossQsigm:0.0598
Episode:195 meanR:193.9600 rate:0.2960 gloss:0.2070 dloss:6.1315 dlossR:0.4926 dlossQ:1.7563 dlossRsigm:4.3752 dlossQsigm:0.0563
Episode:196 meanR:193.3500 rate:0.3040 gloss:0.3308 dloss:6.6258 dlossR:0.4825 dlossQ:1.9790 dlossRsigm:4.6468 dlossQsigm:0.0583
Episode:197 meanR:191.8700 rate:0.2480 gloss:0.3239 dloss:7.2185 dlossR:0.5637 dlossQ:2.1000 dlossRsigm:5.1185 dlossQsigm:0.0601
Episode:198 meanR:191.2900 rate:0.3440 gloss:0.1450 dloss:6.9490 dlossR:0.4288 dlossQ:2.4476 dlossRsigm:4.5014 dlossQsigm:0.0653
Episode:199 meanR:190.9800 rate:0.3220 gloss:0.0502 dloss:6.9647 dlossR:0.4583 dlossQ:2.2309 dlossRsigm:4.7338 dlossQsigm:0.0628
Episode:200 meanR:190.3400 rate:0.3140 gloss:0.3576 dloss:6.8693 dlossR:0.4691 dlossQ:2.1345 dlos

Episode:258 meanR:152.7300 rate:0.2500 gloss:0.6292 dloss:9.8827 dlossR:0.5621 dlossQ:3.5916 dlossRsigm:6.2910 dlossQsigm:0.0740
Episode:259 meanR:152.9500 rate:0.1660 gloss:0.0574 dloss:12.6212 dlossR:0.6955 dlossQ:3.7641 dlossRsigm:8.8572 dlossQsigm:0.0861
Episode:260 meanR:152.6500 rate:0.2080 gloss:0.1505 dloss:12.5052 dlossR:0.6272 dlossQ:4.0800 dlossRsigm:8.4252 dlossQsigm:0.0884
Episode:261 meanR:152.8500 rate:0.2520 gloss:0.4603 dloss:9.5011 dlossR:0.5592 dlossQ:2.8834 dlossRsigm:6.6177 dlossQsigm:0.0660
Episode:262 meanR:153.5900 rate:0.2940 gloss:0.8816 dloss:10.0293 dlossR:0.4982 dlossQ:3.6720 dlossRsigm:6.3573 dlossQsigm:0.0753
Episode:263 meanR:153.9000 rate:0.2140 gloss:1.0398 dloss:10.9765 dlossR:0.6176 dlossQ:3.7302 dlossRsigm:7.2464 dlossQsigm:0.0794
Episode:264 meanR:153.8600 rate:0.1480 gloss:0.0043 dloss:13.2528 dlossR:0.7259 dlossQ:4.0882 dlossRsigm:9.1646 dlossQsigm:0.0916
Episode:265 meanR:153.2400 rate:0.1140 gloss:0.0140 dloss:12.6104 dlossR:0.7849 dlossQ:3.185

...

Episode:10844 meanR:48.4600 rate:0.1120 gloss:0.0001 dloss:8.5343 dlossR:0.7697 dlossQ:0.4203 dlossRsigm:8.1140 dlossQsigm:0.0421
Episode:10845 meanR:48.6500 rate:0.1860 gloss:0.0000 dloss:6.9228 dlossR:0.6331 dlossQ:0.3525 dlossRsigm:6.5703 dlossQsigm:0.0568
Episode:10846 meanR:48.6200 rate:0.0720 gloss:0.0805 dloss:7.8083 dlossR:0.8245 dlossQ:0.4227 dlossRsigm:7.3856 dlossQsigm:0.0570
Episode:10847 meanR:48.4800 rate:0.0820 gloss:0.0000 dloss:7.6126 dlossR:0.8075 dlossQ:0.3239 dlossRsigm:7.2886 dlossQsigm:0.0564
Episode:10848 meanR:48.6600 rate:0.1100 gloss:0.0214 dloss:8.8927 dlossR:0.7803 dlossQ:0.4139 dlossRsigm:8.4788 dlossQsigm:0.0376
Episode:10849 meanR:48.4600 rate:0.1040 gloss:0.0056 dloss:7.3308 dlossR:0.7412 dlossQ:0.4419 dlossRsigm:6.8888 dlossQsigm:0.0844
Episode:10850 meanR:48.6200 rate:0.1180 gloss:0.0000 dloss:9.3869 dlossR:0.7705 dlossQ:0.4597 dlossRsigm:8.9272 dlossQsigm:0.0360
Episode:10851 meanR:48.8600 rate:0.1620 gloss:0.0003 dloss:8.0678 dlossR:0.6844 dlossQ:0.4

Episode:10910 meanR:49.4100 rate:0.0900 gloss:0.0000 dloss:7.0111 dlossR:0.7620 dlossQ:0.4141 dlossRsigm:6.5970 dlossQsigm:0.0861
Episode:10911 meanR:49.4900 rate:0.0960 gloss:0.0000 dloss:6.6507 dlossR:0.7490 dlossQ:0.4506 dlossRsigm:6.2001 dlossQsigm:0.0886
Episode:10912 meanR:49.9100 rate:0.1520 gloss:0.0002 dloss:6.1903 dlossR:0.6658 dlossQ:0.4308 dlossRsigm:5.7595 dlossQsigm:0.0775
Episode:10913 meanR:49.6600 rate:0.0640 gloss:0.0006 dloss:10.2047 dlossR:0.8510 dlossQ:1.5656 dlossRsigm:8.6391 dlossQsigm:0.0460
Episode:10914 meanR:49.4800 rate:0.0640 gloss:0.1768 dloss:10.1446 dlossR:0.8585 dlossQ:1.8686 dlossRsigm:8.2760 dlossQsigm:0.0407
Episode:10915 meanR:49.2100 rate:0.0760 gloss:0.0000 dloss:8.9276 dlossR:0.8343 dlossQ:0.3689 dlossRsigm:8.5588 dlossQsigm:0.0421
Episode:10916 meanR:49.1900 rate:0.0780 gloss:0.0000 dloss:9.2768 dlossR:0.8315 dlossQ:0.3198 dlossRsigm:8.9570 dlossQsigm:0.0401
Episode:10917 meanR:49.4200 rate:0.1240 gloss:0.0012 dloss:7.3336 dlossR:0.7282 dlossQ:0

Episode:10975 meanR:46.8300 rate:0.0740 gloss:0.0011 dloss:8.5041 dlossR:0.8223 dlossQ:0.3950 dlossRsigm:8.1091 dlossQsigm:0.0553
Episode:10976 meanR:46.6100 rate:0.0740 gloss:0.0000 dloss:7.2451 dlossR:0.8130 dlossQ:0.4037 dlossRsigm:6.8414 dlossQsigm:0.0641
Episode:10977 meanR:46.4900 rate:0.1260 gloss:0.0000 dloss:7.8889 dlossR:0.7360 dlossQ:0.2828 dlossRsigm:7.6061 dlossQsigm:0.0488
Episode:10978 meanR:46.4300 rate:0.0700 gloss:0.0611 dloss:7.5478 dlossR:0.8233 dlossQ:0.9015 dlossRsigm:6.6463 dlossQsigm:0.0612
Episode:10979 meanR:46.1200 rate:0.0780 gloss:0.0000 dloss:8.2635 dlossR:0.8220 dlossQ:0.3026 dlossRsigm:7.9608 dlossQsigm:0.0482
Episode:10980 meanR:46.3100 rate:0.1160 gloss:0.0000 dloss:6.0155 dlossR:0.6882 dlossQ:0.4840 dlossRsigm:5.5315 dlossQsigm:0.1248
Episode:10981 meanR:46.0600 rate:0.0580 gloss:3.2719 dloss:8.1872 dlossR:0.8016 dlossQ:1.3101 dlossRsigm:6.8771 dlossQsigm:0.1159
Episode:10982 meanR:46.2700 rate:0.1060 gloss:0.0207 dloss:6.3341 dlossR:0.6917 dlossQ:0.5

Episode:11041 meanR:45.2700 rate:0.1000 gloss:0.0003 dloss:8.6842 dlossR:0.7987 dlossQ:0.4086 dlossRsigm:8.2756 dlossQsigm:0.0353
Episode:11042 meanR:45.2500 rate:0.0860 gloss:0.0140 dloss:8.8915 dlossR:0.8147 dlossQ:0.5262 dlossRsigm:8.3653 dlossQsigm:0.0443
Episode:11043 meanR:45.3000 rate:0.0980 gloss:0.0058 dloss:7.1714 dlossR:0.7756 dlossQ:0.4580 dlossRsigm:6.7134 dlossQsigm:0.0585
Episode:11044 meanR:45.2000 rate:0.0680 gloss:0.0086 dloss:7.8500 dlossR:0.8041 dlossQ:0.9052 dlossRsigm:6.9448 dlossQsigm:0.1016
Episode:11045 meanR:45.2500 rate:0.1200 gloss:0.0006 dloss:7.6466 dlossR:0.7483 dlossQ:0.3866 dlossRsigm:7.2600 dlossQsigm:0.0490
Episode:11046 meanR:45.0800 rate:0.0640 gloss:0.0023 dloss:10.1497 dlossR:0.8530 dlossQ:1.6937 dlossRsigm:8.4560 dlossQsigm:0.0433
Episode:11047 meanR:45.2100 rate:0.1220 gloss:0.1911 dloss:8.2470 dlossR:0.7419 dlossQ:0.6015 dlossRsigm:7.6455 dlossQsigm:0.0551
Episode:11048 meanR:45.2700 rate:0.1220 gloss:0.0010 dloss:8.6621 dlossR:0.7401 dlossQ:0.

Episode:11106 meanR:48.6500 rate:0.1120 gloss:0.0090 dloss:9.1335 dlossR:0.7660 dlossQ:0.4477 dlossRsigm:8.6858 dlossQsigm:0.0490
Episode:11107 meanR:48.7100 rate:0.0800 gloss:0.0322 dloss:7.8864 dlossR:0.8100 dlossQ:0.3589 dlossRsigm:7.5275 dlossQsigm:0.0547
Episode:11108 meanR:48.7900 rate:0.1360 gloss:0.0002 dloss:8.0728 dlossR:0.7165 dlossQ:0.3597 dlossRsigm:7.7131 dlossQsigm:0.0517
Episode:11109 meanR:49.0700 rate:0.1460 gloss:0.0459 dloss:6.6321 dlossR:0.6823 dlossQ:0.3916 dlossRsigm:6.2404 dlossQsigm:0.0702
Episode:11110 meanR:48.9200 rate:0.0820 gloss:0.0001 dloss:7.2224 dlossR:0.7898 dlossQ:0.3305 dlossRsigm:6.8919 dlossQsigm:0.0717


# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(g_loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')

In [ ]:
eps, arr = np.array(d_loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

In [ ]:
eps, arr = np.array(d_lossR_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses R')

In [ ]:
eps, arr = np.array(d_lossQ_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses Q')

## Testing

Let's checkout how our trained agent plays the game.

In [34]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(100):
    #while True:
        state = env.reset()
        total_reward = 0

        # Steps/batches
        #for _ in range(111111111111111111):
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: np.reshape(state, [1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                break
        # Print and break condition
        print('total_reward: {}'.format(total_reward))
        # if total_reward == 500:
        #     break
                
# Closing the env
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.